In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
df_road_network = pd.read_csv('./data/road_network_sub-dataset.csv',sep='\t')
df_road_network

,link_id,width,direction,snodeid,enodeid,length,speedclass,lanenum
0,1562548955,30,3,1520445066,1549742690,0.038,6,1
1,1570012174,55,2,1520482305,1554157827,0.013,5,2
2,1542179399,30,2,1520491527,1531173821,0.146,6,1
3,1562548950,55,1,1520433538,1549742156,0.029,7,1
4,1526068509,30,2,1520486391,1520499209,0.156,6,1
...,...,...,...,...,...,...,...,...
45143,1542174850,55,1,1531470215,1531432689,0.028,7,1
45144,1526066032,30,2,1520457318,1520455758,0.112,6,1
45145,1542091409,55,1,1520434054,1530823987,0.083,7,1
45146,1525859466,55,2,1520406288,1520411419,0.020,5,2


In [3]:
df_road_network['direction'].isnull().any()

False

In [4]:
list(df_road_network.link_id)[45137:]

[1525821388,
 1539164453,
 1541615435,
 1542124325,
 1561692469,
 1526066036,
 1542174850,
 1526066032,
 1542091409,
 1525859466,
 1568519120]

In [13]:
len(road_network_linkid_filtered)

NameError: name 'road_network_linkid_filtered' is not defined

In [6]:
speed = pd.read_pickle('./data/event_traffic_beijing_1km_mv_avg_15min_completion.pkl')

In [11]:
speed_filter = []
for i in road_network_linkid_filtered:
    speed_filter.append(speed[str(i)])
print(np.array(speed_filter).shape)

(1448, 5856)


In [12]:
list(speed.items())[:5]

[('1562548955',
  array([41.34806872, 37.24135644, 33.83275   , ..., 45.48708111,
         39.84807422, 42.93081685])),
 ('1570012174',
  array([53.26809817, 39.71741952, 39.65570926, ..., 30.973098  ,
         29.40824823, 29.00411736])),
 ('1542179399',
  array([38.3801296 , 45.98615911, 45.91535089, ..., 33.80820543,
         35.5688108 , 33.45157817])),
 ('1562548950',
  array([27.23629054, 36.68858464, 43.63393038, ..., 42.63756878,
         43.67184775, 44.59122234])),
 ('1526068509',
  array([50.20075203, 47.35078   , 44.63151222, ..., 50.12629044,
         48.13684422, 42.4951174 ]))]

In [13]:
def compute_graph_sml(data, delta):
    # (num_nodes,total_intervals*num_features)
    # n=num_nodes, c = total_intervals*num_features
    n, c = data.shape  
    graph_sml = np.zeros((n, n), dtype=np.float32)
    for i in range(n):
        for j in range(i, n):
            a = np.linalg.norm(data[i] - data[j])**2
            b = np.minimum(np.linalg.norm(data[i])**2, np.linalg.norm(data[j])**2)
            c = np.exp(-a / b)
            if c > delta:
                graph_sml[j, i] = graph_sml[i, j] = c

    return graph_sml

In [14]:
sml_mat = compute_graph_sml(np.array(speed_filter),0.1)
sml_mat.shape

(1448, 1448)

In [15]:
with open('./data/sml_mat.pkl', 'wb') as file:
    pickle.dump(sml_mat, file)
file.close()
sml_mat = pd.read_pickle('./data/sml_mat.pkl')
sml_mat

array([[1.        , 0.9578839 , 0.90765107, ..., 0.9598526 , 0.93288016,
        0.94673467],
       [0.9578839 , 1.        , 0.83128476, ..., 0.91557604, 0.86159635,
        0.96346533],
       [0.90765107, 0.83128476, 1.        , ..., 0.9417208 , 0.941189  ,
        0.78483236],
       ...,
       [0.9598526 , 0.91557604, 0.9417208 , ..., 1.        , 0.96195155,
        0.87149876],
       [0.93288016, 0.86159635, 0.941189  , ..., 0.96195155, 1.        ,
        0.81982285],
       [0.94673467, 0.96346533, 0.78483236, ..., 0.87149876, 0.81982285,
        1.        ]], dtype=float32)

In [17]:
a=1
b=2
c=3
if a==1 or b==2 or c==3:
    print('yes')

yes


In [12]:
df_road_network.loc[1,'direction']

2

In [5]:
adj_matrix = np.zeros((45148, 45148))
# 遍历列 A 的数据
for index, row in df_road_network.iterrows():
    current_value1 = row['snodeid']
    current_value2 = row['enodeid']         
    duplicate_indices_pre = df_road_network[df_road_network['enodeid'] == current_value1].index.tolist()
    for adj_ind in set(duplicate_indices_pre):
        adj_ind_dir = df_road_network.loc[adj_ind,'direction']
        if row['direction'] == 0 or row['direction'] == 1:
            if adj_ind_dir==0 or adj_ind_dir==1:
                adj_matrix[index][adj_ind] = 1
                adj_matrix[adj_ind][index] = 1
            elif adj_ind_dir==2:
                adj_matrix[adj_ind][index] = 1
            else:
                adj_matrix[index][adj_ind] = 1   
        elif row['direction'] == 2:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==2:
                adj_matrix[adj_ind][index] = 1
        else:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==3:
                adj_matrix[index][adj_ind] = 1
                
    duplicate_indices_post = df_road_network[df_road_network['snodeid'] == current_value2].index.tolist()
    for adj_ind in set(duplicate_indices_post):
        adj_ind_dir = df_road_network.loc[adj_ind,'direction']
        if row['direction'] == 0 or row['direction'] == 1:
            if adj_ind_dir==0 or adj_ind_dir==1:
                adj_matrix[index][adj_ind] = 1
                adj_matrix[adj_ind][index] = 1
            elif adj_ind_dir==2:
                adj_matrix[index][adj_ind] = 1
            else:
                adj_matrix[adj_ind][index] = 1   
        elif row['direction'] == 2:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==2:
                adj_matrix[index][adj_ind] = 1
        else:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==3:
                adj_matrix[adj_ind][index] = 1
adj_matrix.shape     

(45148, 45148)

In [8]:
len(adj_matrix[adj_matrix==1])

57859

In [51]:
# 依据conn邻接矩阵生成eigenmaps,用于生成空间嵌入
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.linalg import eigh
def compute_eigenmaps(adj_mx, k):
    A = adj_mx.copy()
    # row, col = A.nonzero()
    # A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

    n_components,labels = connected_components(csr_matrix(A), directed=True, return_labels=True)
    assert n_components == 1  # the graph should be connected

    n = A.shape[0]
    A = zero_diagonals(A)
    # degrees = np.sum(A, axis=1)
    # # 避免除以零，将度为零的节点的度设置为一个小正数（例如1e-6）
    # degrees[degrees == 0] = 1e-6
    # D = degrees**(-1/2)
    # L = np.eye(n) - (A * D).T * D  # normalized Laplacian
    D = np.sum(A, axis=1)**(-1/2)
    L = np.eye(n) - (A * D).T * D  # normalized Laplacian

    _, v = eigh(L)
    print('pre_eigenmaps.shape:',v.shape)
    eigenmaps = v[:, 1:(k + 1)]  # eigenvectors corresponding to the k smallest non-trivial eigenvalues

    return eigenmaps

def zero_diagonals(x):
    y = x.copy()
    y[np.diag_indices_from(y)] = 0

    return y

In [52]:
adj_matrix_filtered.shape

(1448, 1448)

In [ ]:
eigenmaps = compute_eigenmaps(adj_matrix_filtered,8)
eigenmaps.shape

In [ ]:
with open('./data/eigenmaps.pkl', 'wb') as file:
    pickle.dump(eigenmaps, file)
file.close()

In [ ]:
A = adj_matrix.copy()
# row, col = A.nonzero()
# A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

n_components,labels = connected_components(csr_matrix(A), directed=True, return_labels=True)
print(n_components)
print(labels)

In [24]:
# 使用unique函数获取唯一的值以及它们的计数
unique_values, counts = np.unique(labels, return_counts=True)
unique_values

array([   0,    1,    2, ..., 1436, 1437, 1438], dtype=int32)

In [36]:
sl = sorted(list(counts),reverse=True)
sl[:10]

[11129, 7577, 6182, 1448, 1365, 783, 715, 585, 495, 465]

In [48]:
labels

array([  0,   1,   1, ...,   7, 886,   6], dtype=int32)

In [44]:
len(counts)

1439

In [51]:
counts[20]

1448

In [7]:
indices = np.where(labels == 20)
indices_ = indices[0]
indices_

NameError: name 'labels' is not defined

In [55]:
len(indices_)

1448

In [58]:
indices_[2]

260

In [50]:
np.where(counts==1448)

(array([20]),)

In [ ]:
# 获取数量最多的值和其出现次数
most_frequent_value = unique_values[3]
count_of_most_frequent_value = counts[3]
print(most_frequent_value)
print(count_of_most_frequent_value)

3
27


In [11]:
# 使用unique函数获取唯一的值以及它们的计数
unique_values, counts = np.unique(labels, return_counts=True)



In [71]:
indices = np.where(labels == 20)
indices_ = indices[0]
# 使用索引数组从大矩阵中取出行和列
adj_matrix_filtered = adj_matrix[indices_][:, indices_]
adj_matrix_filtered.shape

(1448, 1448)

In [61]:
len(adj_matrix_filtered[adj_matrix_filtered==1])

1987

In [16]:
indices = np.where(labels == 20)
len(indices)

(array([    0,    59,   109, ..., 44984, 44997, 45118]),)

In [62]:
len(indices[0])

1448

In [ ]:
with open('./data/indices.pkl', 'wb') as file:
    pickle.dump(indices[0], file)
file.close()

In [38]:
with open('./data/indices.pkl', 'wb') as file:
    pickle.dump(indices[0], file)
file.close()

In [37]:
len(indices[0])

2307

In [39]:
indices = pd.read_pickle('./data/indices.pkl')
len(indices)

2307

In [40]:
road_network_linkid_filtered = df_road_network.loc[indices, 'link_id']
len(road_network_linkid_filtered)

2307

In [43]:
with open('./data/road_network_linkid_filtered.pkl', 'wb') as file:
    pickle.dump(list(road_network_linkid_filtered), file)
file.close()
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered.pkl')
len(road_network_linkid_filtered)

2307

In [15]:
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered.pkl')
road_length = np.zeros((2307, 1))
j = 0
for i in road_network_linkid_filtered:
    filtered_df = df_road_network[df_road_network['link_id'] == i]
    print(filtered_df)
    length = filtered_df['length']
    road_length[j][0] = length
    j += 1
# with open('./data/road_filtered_length.pkl', 'wb') as file:
#     pickle.dump(road_length, file)

       link_id  width  direction     snodeid     enodeid  length  speedclass  \
10  1526063771     55          1  1520458585  1520465312   0.018           7   

    lanenum  
10        1  
       link_id  width  direction     snodeid     enodeid  length  speedclass  \
12  1526063773    130          2  1520471556  1520467910   0.017           5   

    lanenum  
12        3  
        link_id  width  direction     snodeid     enodeid  length  speedclass  \
135  1538291413     55          2  1529712794  1520463549   0.012           6   

     lanenum  
135        2  
        link_id  width  direction     snodeid     enodeid  length  speedclass  \
138  1526061840     30          2  1520467734  1520457562   0.006           6   

     lanenum  
138        1  
        link_id  width  direction     snodeid     enodeid  length  speedclass  \
144  1526061849     55          2  1520465718  1520466756   0.028           6   

     lanenum  
144        2  
        link_id  width  direction     snode

KeyboardInterrupt: 

In [11]:
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered.pkl')
len(road_network_linkid_filtered)

1448

In [12]:
adj_matrix_filtered = pd.read_pickle('./data/adj_matrix_filtered.pkl')
len(adj_matrix_filtered)

1448

In [17]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.linalg import eigh
C = adj_matrix.copy()
# row, col = A.nonzero()
# A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

n_componentsC,labelsC = connected_components(csr_matrix(C), directed=True, connection='strong', return_labels=True)
print(n_componentsC)
print(len(labelsC))

20849
45148


In [18]:
labelsC

array([    0,    49,  1298, ...,  4160, 16956,  3003], dtype=int32)

In [19]:
# 使用unique函数获取唯一的值以及它们的计数
unique_values, counts = np.unique(labelsC, return_counts=True)
unique_values

array([    0,     1,     2, ..., 20846, 20847, 20848], dtype=int32)

In [25]:
len(counts)

20849

In [21]:
sl = sorted(list(counts),reverse=True)
sl[:10]

[6392, 3848, 2307, 600, 537, 432, 278, 232, 231, 218]

In [22]:
np.where(counts==2307)

(array([2040]),)

In [26]:
indices = np.where(labelsC == 2040)
indices

(array([   10,    12,   135, ..., 45078, 45093, 45127]),)

In [27]:
len(indices[0])

2307

In [28]:
# 使用索引数组从大矩阵中取出行和列
indices_ = indices[0]
adj_matrix_filteredC = adj_matrix[indices_][:, indices_]
adj_matrix_filteredC.shape

(2307, 2307)

In [32]:
D = adj_matrix_filteredC.copy()
# row, col = A.nonzero()
# A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

n_componentsD,labelsD = connected_components(csr_matrix(D), directed=True, connection='strong', return_labels=True)
print(n_componentsD)
print(len(labelsD))

1
2307


In [33]:
labelsD

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [36]:
len(indices_)

2307

In [44]:
with open('./data/adj_matrix_filtered.pkl', 'wb') as file:
    pickle.dump(adj_matrix_filteredC, file)
file.close()

In [45]:
adj_matrix_filteredC.shape

(2307, 2307)

In [31]:
len(adj_matrix_filteredC[adj_matrix_filteredC==1])

2968

In [74]:
B = adj_matrix_filtered.copy()
# row, col = A.nonzero()
# A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

n_componentsB,labelsB = connected_components(csr_matrix(B), directed=True, return_labels=True)
print(n_componentsB)
print(len(labelsB))

1
1448


In [75]:
labelsB

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [80]:
has_nan = np.isnan(adj_matrix_filtered).any()
has_inf = np.isinf(adj_matrix_filtered).any()
if has_nan or has_inf:
    # 处理包含NaN或无穷大值的矩阵
    print('yes')
else:
    # 继续进行特征值计算
    print('no')

no


In [89]:
eigenmaps = compute_eigenmaps(adj_matrix_filtered,8)
eigenmaps

pre_eigenmaps.shape: (1448, 1448)


array([[-2.50056326e-48, -1.35987103e-50, -5.32970185e-53, ...,
         6.26053539e-50, -1.87347716e-49, -1.27640520e-45],
       [-2.42212988e-47, -1.77489875e-50,  9.56857315e-53, ...,
        -8.29088491e-50, -1.59922449e-49, -7.12465194e-46],
       [ 6.59495664e-19, -2.56273968e-19,  7.38243033e-21, ...,
        -8.28610693e-19, -4.12081479e-18, -4.70230758e-18],
       ...,
       [ 3.23271965e-46,  1.41376957e-49, -1.12456341e-50, ...,
         2.37672950e-24,  1.51258608e-18,  1.54165363e-18],
       [-4.26825248e-22,  1.05001912e-31, -1.36848518e-32, ...,
        -5.53342623e-21,  1.98082814e-19,  2.58104156e-18],
       [-3.86242597e-18, -2.93393384e-19, -5.16690098e-20, ...,
        -3.43753245e-20, -8.38210386e-20, -2.11520153e-18]])

In [91]:
with open('./data/eigenmaps.pkl', 'wb') as file:
    pickle.dump(eigenmaps, file)
file.close()
with open('./data/adj_matrix_filtered.pkl', 'wb') as file:
    pickle.dump(adj_matrix_filtered, file)
file.close()

In [90]:
eigenmaps.shape

(1448, 8)

In [46]:
df_pos_mercator = pd.read_csv('./data/event_link_set_all_beijing_1km_filtered.txt', sep='\t', header=None)
df_pos_mercator.columns = ['link_id','longitude','latitude']
df_pos_mercator

,link_id,longitude,latitude
0,1562548955,1.295412e+07,4.823689e+06
1,1570012174,1.295539e+07,4.832144e+06
2,1542179399,1.295908e+07,4.836610e+06
3,1562548950,1.294295e+07,4.814772e+06
4,1526068509,1.295517e+07,4.836727e+06
...,...,...,...
45143,1542174850,1.296855e+07,4.836903e+06
45144,1526066032,1.296053e+07,4.821160e+06
45145,1542091409,1.294552e+07,4.820323e+06
45146,1525859466,1.295354e+07,4.829609e+06


In [10]:
(df_pos_mercator['link_id'] == df_road_network['link_id'])

0        True
1        True
2        True
3        True
4        True
         ... 
45143    True
45144    True
45145    True
45146    True
45147    True
Name: link_id, Length: 45148, dtype: bool

In [48]:
# 使用not来检查是否存在False值
if not all((df_pos_mercator['link_id'] == df_road_network['link_id']).tolist()):
    print("列表中存在False值")
else:
    print("列表中没有False值")

列表中没有False值


In [47]:
df_pos_mercator[df_pos_mercator.link_id.duplicated()==True]

,link_id,longitude,latitude


In [99]:
indices_

array([   59,   109,   260, ..., 44984, 44997, 45118])

In [94]:
# distance tool
def eucl_distance(x, y):
    return np.linalg.norm(x - y)


def get_distance_diff(coordinates_mercator):
    temp_result = []
    coordinates_mercator_pre = [coordinates_mercator[0]] + coordinates_mercator[:-1]
    for i, item in enumerate(coordinates_mercator):
        dis_temp = eucl_distance(np.array(item), np.array(coordinates_mercator_pre[i]))
        temp_result.append(dis_temp)
    return temp_result
num_nodes = eigenmaps.shape[0]
dis_mat = np.zeros((num_nodes, num_nodes))

In [95]:
dis_mat.shape

(1448, 1448)

In [103]:
eigenmaps.shape[0]

1448

In [115]:
indices_

array([   59,   109,   260, ..., 44984, 44997, 45118])

In [113]:
df_pos_mercator.iloc[45118,1:]

longitude    1.295482e+07
latitude     4.825505e+06
Name: 45118, dtype: float64

In [12]:
indices = pd.read_pickle('./data/indices.pkl')

In [49]:
len(indices)

2307

In [132]:
dis_mat_

array([[   0.        , 2342.08285523, 1405.34887242, ..., 3662.47737973,
         416.17633323,  394.13558933],
       [2342.08285523,    0.        , 2033.03027001, ..., 1483.30869575,
        2465.80385071, 2101.68279225],
       [1405.34887242, 2033.03027001,    0.        , ..., 3515.73637184,
        1136.05888948, 1573.22364082],
       ...,
       [3662.47737973, 1483.30869575, 3515.73637184, ...,    0.        ,
        3861.4697726 , 3348.2878564 ],
       [ 416.17633323, 2465.80385071, 1136.05888948, ..., 3861.4697726 ,
           0.        ,  784.48306469],
       [ 394.13558933, 2101.68279225, 1573.22364082, ..., 3348.2878564 ,
         784.48306469,    0.        ]])

In [50]:
num_nodes = 2307
dis_mat = np.zeros((num_nodes, num_nodes))
m,n= 0,0
for i in indices:
    n=0
    for j in indices:
        if i!=j:
            dis_mat[m][n] = np.linalg.norm(df_pos_mercator.iloc[i,1:] - df_pos_mercator.iloc[j,1:])
        n+=1
    m+=1
dis_mat.shape

(2307, 2307)

In [53]:
dis_mat

array([[   0.        , 1439.98661938, 1942.77527716, ..., 1928.43715102,
        3250.96383137, 6228.51999247],
       [1439.98661938,    0.        , 2664.91755478, ..., 2649.14432981,
        3852.10803347, 6657.80349945],
       [1942.77527716, 2664.91755478,    0.        , ...,   15.87463288,
        1308.46921489, 4287.19058949],
       ...,
       [1928.43715102, 2649.14432981,   15.87463288, ...,    0.        ,
        1323.01148588, 4301.21599464],
       [3250.96383137, 3852.10803347, 1308.46921489, ..., 1323.01148588,
           0.        , 2983.77453231],
       [6228.51999247, 6657.80349945, 4287.19058949, ..., 4301.21599464,
        2983.77453231,    0.        ]])

In [54]:
with open('./data/dis_mat.pkl', 'wb') as file:
    pickle.dump(dis_mat, file)
file.close()

In [137]:
with open('./data/dis_mat.pkl', 'wb') as file:
    pickle.dump(dis_mat, file)
with open('./data/eigenmaps.pkl', 'wb') as file:
    pickle.dump(eigenmaps, file)
with open('./data/adj_matrix_filtered.pkl', 'wb') as file:
    pickle.dump(adj_matrix_filtered, file)

In [136]:
adj_matrix_filtered.shape

(1448, 1448)

In [5]:
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered.pkl')
len(road_network_linkid_filtered)

1448

In [7]:
len(road_network_linkid_filtered)

1448

In [73]:
adj_matrix_filtered.shape

(1556, 1556)

In [16]:
eigenmaps_ = pd.read_pickle('./data/eigenmaps.pkl')
eigenmaps_.shape

(1448, 8)

In [76]:
dis_mat_ = pd.read_pickle('./data/dis_mat.pkl')
dis_mat_

array([[2224.60291078,  310.47823588, 2113.43622977, ..., 2397.19367017,
        1945.80687839,    0.        ],
       [2224.60291078, 2342.08285523, 1405.34887242, ...,  416.17633323,
         394.13558933,    0.        ],
       [ 310.47823588, 2342.08285523, 2033.03027001, ..., 2465.80385071,
        2101.68279225,    0.        ],
       ...,
       [1468.91486721, 3662.47737973, 1483.30869575, ..., 3861.4697726 ,
        3348.2878564 ,    0.        ],
       [2397.19367017,  416.17633323, 2465.80385071, ..., 3861.4697726 ,
         784.48306469,    0.        ],
       [1945.80687839,  394.13558933, 2101.68279225, ..., 3348.2878564 ,
         784.48306469,    0.        ]])

In [77]:
dis_mat_.shape

(1556, 1556)

In [63]:
dis_mat = np.zeros((num_nodes, num_nodes))
dis_mat.shape

(1556, 1556)

In [57]:

df_pos_mercator.iloc[0,1:] - df_pos_mercator.iloc[1,1:]

longitude   -1267.75350
latitude    -8454.80502
dtype: float64

In [58]:
np.linalg.norm(df_pos_mercator.iloc[0,1:] - df_pos_mercator.iloc[1,1:])

8549.323181572685

In [ ]:
has_nan = np.isnan(L).any()
has_inf = np.isinf(L).any()
if has_nan or has_inf:
    # 处理包含NaN或无穷大值的矩阵
    print('has_nan yes')
if has_inf:
    print('has_inf yes')   
else:
    # 继续进行特征值计算
    print('no')

In [15]:
ind = pd.read_pickle('./data/indices.pkl')
road_id = pd.read_pickle('./data/road_network_linkid_filtered.pkl')
ind[0]

59

In [21]:
j = 0
road_length = np.zeros((1448,2))
for i in ind:
    row = df_road_network.iloc[i]
    length = row['length']
    road_length[j][0] = road_id[j]
    road_length[j][1] = length
    j += 1
road_length

array([[1.52598295e+09, 6.40000000e-02],
       [1.56972052e+09, 5.20000000e-02],
       [1.52608467e+09, 5.00000000e-03],
       ...,
       [1.56180614e+09, 7.00000000e-03],
       [1.52602014e+09, 4.30000000e-02],
       [1.56972105e+09, 1.05000000e-01]])

In [22]:
with open('./data/road_filtered_length.pkl', 'wb') as file:
    pickle.dump(road_length, file)

In [1]:
import pandas as pd

In [2]:
inputs = pd.read_pickle('./data/inputs.pkl')
inputs.shape

(5795, 1, 1448, 1)

In [3]:
targets = pd.read_pickle('./data/targets.pkl')
targets.shape

(5795, 1, 1448, 1)

In [44]:
extras = pd.read_pickle('./data/extras.pkl')
extras

(array([[ 1.],
        [ 2.],
        [ 3.],
        ...,
        [93.],
        [94.],
        [95.]]),
 array([[ 2.],
        [ 3.],
        [ 4.],
        ...,
        [94.],
        [95.],
        [96.]]),
 array([[ 1.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  2.,  0.],
        [ 1.,  0.,  0.,  0.,  3.,  0.],
        ...,
        [ 1.,  0.,  0., 23.,  1.,  0.],
        [ 1.,  0.,  0., 23.,  2.,  0.],
        [ 1.,  0.,  0., 23.,  3.,  0.]]),
 array([[ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  2.,  0.],
        ...,
        [ 1.,  0.,  0., 23.,  0.,  0.],
        [ 1.,  0.,  0., 23.,  1.,  0.],
        [ 1.,  0.,  0., 23.,  2.,  0.]]))

In [16]:
extras[2][:10]

array([[1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 2., 0.],
       [1., 0., 0., 0., 3., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 2., 0.],
       [1., 0., 0., 1., 3., 0.],
       [1., 0., 0., 2., 0., 0.],
       [1., 0., 0., 2., 1., 0.],
       [1., 0., 0., 2., 2., 0.]])

In [15]:
extras[3][:10]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 2., 0.],
       [1., 0., 0., 0., 3., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 2., 0.],
       [1., 0., 0., 1., 3., 0.],
       [1., 0., 0., 2., 0., 0.],
       [1., 0., 0., 2., 1., 0.]])

In [10]:
extras[0]

array([[ 1.],
       [ 2.],
       [ 3.],
       ...,
       [93.],
       [94.],
       [95.]])

In [11]:
data_dict = {'x':inputs,'y':targets,'xtime0':extras[0],'ytime0':extras[1],'xtime1':extras[2],'ytime1':extras[3]}

In [12]:
data_dict

{'x': array([[[[39.49011238],
          [41.71097671],
          [21.95539157],
          ...,
          [37.80744096],
          [31.7748215 ],
          [41.3237378 ]]],
 
 
        [[[37.60468133],
          [39.54602422],
          [26.69643359],
          ...,
          [38.61736978],
          [30.14179661],
          [41.30677289]]],
 
 
        [[[35.41421356],
          [39.17835533],
          [29.55599136],
          ...,
          [37.416088  ],
          [33.20335906],
          [41.512298  ]]],
 
 
        ...,
 
 
        [[[31.94697511],
          [25.56193889],
          [29.58622667],
          ...,
          [22.24051267],
          [23.58836911],
          [37.32236667]]],
 
 
        [[[32.918958  ],
          [22.01653356],
          [28.54956889],
          ...,
          [24.79648689],
          [26.40292978],
          [37.90195333]]],
 
 
        [[[ 0.        ],
          [ 0.        ],
          [ 0.        ],
          ...,
          [ 0.        ],
        

In [24]:
import numpy as np

data_dict = {'x':inputs,'y':targets,'xtime0':extras[0],'ytime0':extras[1],'xtime1':extras[3],'ytime1':extras[2]}
# 假设 inputs 和 targets 是你的数据
inputs_ = np.array(data_dict['x'])
targets_ = np.array(data_dict['y'])
xtime0_ = np.array(data_dict['xtime0'])
ytime0_ = np.array(data_dict['ytime0'])
xtime1_ = np.array(data_dict['xtime1'])
ytime1_ = np.array(data_dict['ytime1'])

# 获取随机排列的索引
permutation = np.random.permutation(len(inputs_))

# 使用这些索引来重新排列 inputs 和 targets
shuffled_inputs = inputs_[permutation]
shuffled_targets = targets_[permutation]
shuffled_xt0 = xtime0_[permutation]
shuffled_yt0 = ytime0_[permutation]
shuffled_xt1 = xtime1_[permutation]
shuffled_yt1 = ytime1_[permutation]

# 创建一个新的 data_dict
shuffled_data_dict = {'x': shuffled_inputs, 'y': shuffled_targets,'xtime0':shuffled_xt0,'ytime0':shuffled_yt0,'xtime1':shuffled_xt1,'ytime1':shuffled_yt1 }


In [25]:
shuffled_data_dict

{'x': array([[[[34.13640956],
          [42.48167222],
          [31.56690933],
          ...,
          [30.91862667],
          [32.694744  ],
          [40.61762089]]],
 
 
        [[[24.34882022],
          [35.39002044],
          [23.98355178],
          ...,
          [24.87964267],
          [23.83062289],
          [31.03284978]]],
 
 
        [[[29.67567462],
          [32.08209967],
          [25.56151421],
          ...,
          [27.22521409],
          [30.03443195],
          [32.44924001]]],
 
 
        ...,
 
 
        [[[30.563002  ],
          [35.96616778],
          [28.07180089],
          ...,
          [26.389204  ],
          [24.339514  ],
          [31.51547289]]],
 
 
        [[[33.80395673],
          [35.28796378],
          [30.52990797],
          ...,
          [33.15421809],
          [18.6       ],
          [39.36121622]]],
 
 
        [[[30.01054244],
          [36.97795356],
          [27.75198644],
          ...,
          [24.50885444],
        

In [26]:
total_samples = len(shuffled_data_dict['x'])
total_samples

5795

In [27]:
# 假设你有已打乱的数据和对应的目标
# shuffled_data_dict 包含 {'x': shuffled_inputs, 'y': shuffled_targets}

# 定义训练、验证和测试数据的比例
train_ratio = 0.7  # 70% 数据用于训练
val_ratio = 0.15  # 15% 数据用于验证
test_ratio = 0.15  # 15% 数据用于测试

# 计算数据集的大小
total_samples = len(shuffled_data_dict['x'])
train_size = int(train_ratio * total_samples)
val_size = int(val_ratio * total_samples)

# 划分数据集
train_data = {'x': shuffled_data_dict['x'][:train_size], 'y': shuffled_data_dict['y'][:train_size],
              'xtime0':shuffled_data_dict['xtime0'][:train_size],'ytime0':shuffled_data_dict['ytime0'][:train_size],
              'xtime1':shuffled_data_dict['xtime1'][:train_size],'ytime1':shuffled_data_dict['ytime1'][:train_size]}
val_data = {'x': shuffled_data_dict['x'][train_size:train_size + val_size], 'y': shuffled_data_dict['y'][train_size:train_size + val_size],
              'xtime0':shuffled_data_dict['xtime0'][train_size:train_size + val_size],'ytime0':shuffled_data_dict['ytime0'][train_size:train_size + val_size],
              'xtime1':shuffled_data_dict['xtime1'][train_size:train_size + val_size],'ytime1':shuffled_data_dict['ytime1'][train_size:train_size + val_size]}
test_data = {'x': shuffled_data_dict['x'][train_size + val_size:], 'y': shuffled_data_dict['y'][train_size + val_size:],
              'xtime0':shuffled_data_dict['xtime0'][train_size + val_size:],'ytime0':shuffled_data_dict['ytime0'][train_size + val_size:],
              'xtime1':shuffled_data_dict['xtime1'][train_size + val_size:],'ytime1':shuffled_data_dict['ytime1'][train_size + val_size:]}

# 现在，train_data、val_data 和 test_data 包含相应部分的数据


In [19]:
train_data

NameError: name 'train_data' is not defined

In [30]:
len(val_data['x'])

869

In [31]:
len(test_data['x'])

870

In [67]:
import pickle
with open('./data/train.pkl', 'wb') as file:
    pickle.dump(train_data, file)
file.close()
with open('./data/val.pkl', 'wb') as file:
    pickle.dump(val_data, file)
file.close()
with open('./data/test.pkl', 'wb') as file:
    pickle.dump(test_data, file)
file.close()

In [55]:
train_data = pd.read_pickle('./data/train.pkl')
val_data = pd.read_pickle('./data/val.pkl')
test_data = pd.read_pickle('./data/test.pkl')

In [23]:
len(train_data['x'])

4056

In [56]:
train_data['ytime0'] -= 1

In [59]:
train_data['xtime0'] -= 1

In [61]:
val_data['xtime0'] -= 1
val_data['ytime0'] -= 1
test_data['xtime0'] -= 1
test_data['ytime0'] -= 1


In [66]:
set(test_data['xtime0'].reshape(870))

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0}

In [123]:
len_train = len(train_data['x'])
n1 = [train_data['xtime1'][:,i] for i in range(6)]
n2 = [train_data['ytime1'][:,i] for i in range(6)]
i1,i2,i3,i4,i5,i6 = [np.array(n1).reshape(6,len_train,1)[i] for i in range(6)]
t1,t2,t3,t4,t5,t6 = [np.array(n2).reshape(6,len_train,1)[i] for i in range(6)]
len(t5)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
train_data['xtime1'] = i1 
train_data['xtime2'] = i2
train_data['xtime3'] = i3 
train_data['xtime4'] = i4 
train_data['xtime5'] = i5 
train_data['xtime6'] = i6 
train_data['ytime1'] = t1 
train_data['ytime2'] = t2
train_data['ytime3'] = t3 
train_data['ytime4'] = t4 
train_data['ytime5'] = t5 
train_data['ytime6'] = t6 

In [119]:
len_val = len(val_data['x'])
n1 = [val_data['xtime1'][:,i] for i in range(6)]
n2 = [val_data['ytime1'][:,i] for i in range(6)]
i1,i2,i3,i4,i5,i6 = [np.array(n1).reshape(6,len_val,1)[i] for i in range(6)]
t1,t2,t3,t4,t5,t6 = [np.array(n2).reshape(6,len_val,1)[i] for i in range(6)]
len(t5)

869

In [120]:
val_data['xtime1'] = i1 
val_data['xtime2'] = i2
val_data['xtime3'] = i3 
val_data['xtime4'] = i4 
val_data['xtime5'] = i5 
val_data['xtime6'] = i6 
val_data['ytime1'] = t1 
val_data['ytime2'] = t2
val_data['ytime3'] = t3 
val_data['ytime4'] = t4 
val_data['ytime5'] = t5 
val_data['ytime6'] = t6 

In [121]:
len_test = len(test_data['x'])
n1 = [test_data['xtime1'][:,i] for i in range(6)]
n2 = [test_data['ytime1'][:,i] for i in range(6)]
i1,i2,i3,i4,i5,i6 = [np.array(n1).reshape(6,len_test,1)[i] for i in range(6)]
t1,t2,t3,t4,t5,t6 = [np.array(n2).reshape(6,len_test,1)[i] for i in range(6)]
len(t5)

870

In [122]:
test_data['xtime1'] = i1 
test_data['xtime2'] = i2
test_data['xtime3'] = i3 
test_data['xtime4'] = i4 
test_data['xtime5'] = i5 
test_data['xtime6'] = i6 
test_data['ytime1'] = t1 
test_data['ytime2'] = t2
test_data['ytime3'] = t3 
test_data['ytime4'] = t4 
test_data['ytime5'] = t5 
test_data['ytime6'] = t6 

In [2]:
import pickle
import pandas as pd
dis_mat = pd.read_pickle('./data/dis_mat.pkl')

In [3]:
dis_mat.shape


(1448, 1448)

In [4]:
dis_mat

array([[2342.08285523, 1405.34887242, 1806.24702495, ...,  416.17633323,
         394.13558933,    0.        ],
       [2342.08285523, 2033.03027001, 1152.56213881, ..., 2465.80385071,
        2101.68279225,    0.        ],
       [1405.34887242, 2033.03027001, 2256.72643603, ..., 1136.05888948,
        1573.22364082,    0.        ],
       ...,
       [3662.47737973, 1483.30869575, 3515.73637184, ..., 3861.4697726 ,
        3348.2878564 ,    0.        ],
       [ 416.17633323, 2465.80385071, 1136.05888948, ..., 3861.4697726 ,
         784.48306469,    0.        ],
       [ 394.13558933, 2101.68279225, 1573.22364082, ..., 3348.2878564 ,
         784.48306469,    0.        ]])

In [126]:
len(test_data['ytime6'])

870

In [68]:
import torch  # 命令行是逐行立即执行的
content = torch.load('./exps/Qtraffic/MGT/E01/output.pth')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
content.shape

torch.Size([870, 1, 1448, 1])

In [71]:
content

tensor([[[[31.3153],
          [37.1256],
          [35.8870],
          ...,
          [28.1879],
          [21.4503],
          [41.4572]]],


        [[[30.5309],
          [37.2000],
          [22.6001],
          ...,
          [28.3145],
          [30.3549],
          [46.2507]]],


        [[[30.2805],
          [31.1398],
          [24.0937],
          ...,
          [26.8437],
          [26.2403],
          [32.3132]]],


        ...,


        [[[30.3800],
          [35.6451],
          [27.8696],
          ...,
          [26.3721],
          [24.3224],
          [31.3116]]],


        [[[33.7988],
          [35.1874],
          [30.3444],
          ...,
          [32.9856],
          [19.0513],
          [39.2772]]],


        [[[29.8410],
          [36.8411],
          [27.5250],
          ...,
          [24.4778],
          [32.5295],
          [32.2425]]]])